In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Insurance Forecast for Medical Cost Personal Datasets using Linear Regression
 
### Can you accurately predict insurance costs?

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv('/kaggle/input/insurance/insurance.csv')

In [ ]:
df.head()

In [ ]:
df.sex.replace('female', 0, inplace = True)
df.sex.replace('male', 1, inplace = True)

In [ ]:
df.smoker.replace('no', 0, inplace = True)
df.smoker.replace('yes', 1, inplace = True)

In [ ]:
df.region.value_counts()

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
sns.catplot(x = 'sex', y = 'charges', data = df, kind = 'box')
# The men have a larger range of charges even though the median for both sexes is around the same. 
#I confirmed in the cell below that it wasn't because more men went to the hospital
# it could be that the range of issues they're dealing with incurred a more varied amount of charges than women.

In [ ]:
df.sex.value_counts()

In [ ]:
sns.catplot(x = 'smoker', y = 'charges', data = df, kind = 'box')

In [ ]:
sns.scatterplot(df.bmi, df.charges, data = df)
# No clear trend, but there are 2 clusters that emerge: the bigger lower bottom and the upper one

In [ ]:
sns.scatterplot(df.bmi, df.charges, data = df, hue = 'smoker')
# There is a definite trend where the smokers are highlighted.

In [ ]:
sns.catplot('children', 'charges', data = df, kind = 'box')

In [ ]:
sns.scatterplot(df.age, df.charges, data = df)
# Generally, the charges are rising with age

In [ ]:
sns.scatterplot(df.age, df.charges, data = df, hue = 'smoker')

In [ ]:
sns.scatterplot(df.age, df.charges, data = df, hue ='bmi')
#Generally, the higher bmi's have higher charges within the same age range

In [ ]:
df.head()

In [ ]:
sns.scatterplot(df.age, df.charges, data = df, hue ='sex')
# I can't tell too much of a  pattern here, where there are children involved.

In [ ]:
sns.scatterplot(df.age, df.charges, data = df, hue ='children')
# I can't tell too much of a  pattern here, where there are children involved.

In [ ]:
sns.scatterplot(df.age, df.charges, data = df, hue ='region')
#There's no pattern where it comes to region, either

In [ ]:
sns.catplot('region', 'charges', data = df, kind = 'box')
# Region doesn't seem to be playing a role

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr())
#There's a stong correlation between smokers and the charges
# Age with the charges , and then bmi with the charges, as well


In [ ]:
# Because children, sex and region do not appear to have a correlation, I'll drop them.
#df.drop(['sex', 'region'], axis = 1, inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate

In [ ]:
x = df.iloc[:, :-1]

In [ ]:
x.head()

In [ ]:
x.drop(['children','sex','region'], axis = 1, inplace = True)

In [ ]:
y = df.iloc[:, -1:]

In [ ]:
y.head()

In [ ]:
x

In [ ]:
#enc = OneHotEncoder()

In [ ]:
#columntransform = ColumnTransformer([('encoder', enc, [3])],remainder = 'passthrough')

In [ ]:
#x = columntransform.fit_transform(x)

In [ ]:
#x

In [ ]:
#x = pd.DataFrame(x)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state = 1, test_size = 0.2)

In [ ]:
lr = LinearRegression()
dr = DecisionTreeRegressor(random_state = 1)
gr = GradientBoostingRegressor(learning_rate = 0.1)
pr = PolynomialFeatures(degree = 2)
mm = MinMaxScaler()
svrl = SVR(kernel = 'linear')
svrr = SVR(kernel = 'rbf')
svrp = SVR(kernel = 'poly')


In [ ]:
pipelr = Pipeline([('scaler', mm), ('linear', lr)])
pipedr = Pipeline([('scaler', mm), ('decision', dr)])
pipegr = Pipeline([('scaler', mm), ('gradient boosting', gr)])
pipepr = Pipeline([('scaler', mm), ('polinomial', pr)])
pipesvrl = Pipeline([('scaler', mm), ('linear', svrl)])
pipesvrp = Pipeline([('scaler', mm),('poly', svrp)])
pipesvrr = Pipeline([('scaler', mm),('rbf', svrr)])

In [ ]:
pipelr.fit(xtrain, ytrain)
pipelr.score(xtrain, ytrain)
pipelr.score(xtest, ytest)

In [ ]:
cv = cross_validate(pipelr,x, y,  cv= 10)
cv['test_score']

In [ ]:
pipedr.fit(xtrain, ytrain)
pipedr.score(xtrain, ytrain)
pipedr.score(xtest, ytest)

In [ ]:
pipegr.fit(xtrain, ytrain)
pipegr.score(xtrain, ytrain)
pipegr.score(xtest, ytest)

In [ ]:
cv = cross_validate(pipegr,x, y,  cv= 10)
cv['test_score'] 

In [ ]:
pipesvrl.fit(xtrain, ytrain)
pipesvrl.score(xtrain, ytrain)
pipesvrl.score(xtest, ytest)

In [ ]:
pipesvrr.fit(xtrain, ytrain)
pipesvrr.score(xtrain, ytrain)
pipesvrr.score(xtest, ytest)

In [ ]:
pipesvrp.fit(xtrain, ytrain)
pipesvrp.score(xtrain, ytrain)
pipesvrp.score(xtest, ytest)

In [ ]:
 
#x = pipepr.fit_transform(x)
#xtrain_poly, xtest_poly, ytrain,ytest = train_test_split(x, y, random_state = 0, test_size = 0.2)
#pipelr.fit(xtrain_poly, ytrain)
#pipelr.score(xtrain_poly, ytrain)
#pipelr.score(xtest_poly, ytrain)

In [ ]:
x_quad = pipepr.fit_transform(x)
xtrainr, xtestr, ytrainr, ytestr = train_test_split(x_quad,y, random_state = 0, test_size = 0.2)

plr = LinearRegression().fit(xtrainr,ytrainr)

Y_train_pred = plr.predict(xtrainr)
Y_test_pred = plr.predict(xtestr)

print(plr.score(xtestr, ytestr))

In [ ]:
cv = cross_validate(plr,x_quad, y,  cv= 10)
cv['test_score']

In [ ]:
estimators = [('gr', gr),('plr', plr)]

In [ ]:
sr = StackingRegressor(estimators=estimators)

In [ ]:
sr.fit(xtrain, ytrain)
sr.score(xtrain, ytrain)
sr.score(xtest, ytest)

In [ ]:
cv = cross_validate(sr,x, y,  cv= 10)
cv['test_score']

**Conclusion: Sex and Region are not playing a role in determining charges!**

In [ ]:
x.bmi
y 


In [ ]:
sns.distplot(y)

In [ ]:
m, b = np.polyfit(x.bmi, y, 1)

In [ ]:
plt.plot(x, y, 'o')
plt.plot(x, m*x + b)